In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from datetime import datetime
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, UpSampling2D, Dropout,BatchNormalization,GlobalAveragePooling2D
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [2]:
# ! git config --global --unset https.proxy
! git clone https://github.com/seshuad/IMagenet
! ls 'IMagenet/tiny-imagenet-200/'

import time
import imageio.v3 as nd
import numpy as np

path = 'IMagenet/tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open( path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict

def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open( path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [nd.imread( path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB') for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open( path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(nd.imread( path + 'val/images/{}'.format(img_name) ,mode='RGB'))
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    print('finished loading data, in {} seconds'.format(time.time() - t))
    return np.array(train_data).astype('float32') / 255.0, np.array(train_labels), np.array(test_data).astype('float32') / 255.0, np.array(test_labels)

train_images, train_labels, test_images, test_labels = get_data(get_id_dictionary())

print( "train data shape: ",  train_images.shape )
print( "train label shape: ", train_labels.shape )
print( "test data shape: ",   test_images.shape )
print( "test_labels.shape: ", test_labels.shape )

train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.05, random_state=42)

Cloning into 'IMagenet'...
remote: Enumerating objects: 120594, done.
remote: Total 120594 (delta 0), reused 0 (delta 0), pack-reused 120594
Receiving objects: 100% (120594/120594), 212.68 MiB | 39.73 MiB/s, done.
Resolving deltas: 100% (1115/1115), done.
Updating files: 100% (120206/120206), done.
test  train  val  wnids.txt  words.txt
starting loading data
finished loading data, in 46.55139660835266 seconds
train data shape:  (100000, 64, 64, 3)
train label shape:  (100000, 200)
test data shape:  (10000, 64, 64, 3)
test_labels.shape:  (10000, 200)


In [4]:
base_model = tf.keras.models.load_model('/content/model_configuration3.h5')

# Check its architecture
base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 2, 2, 128)         0

In [5]:
base_model.pop()

In [6]:
base_model.trainable=False

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

model = tf.keras.models.Sequential()
model.add(MaxPooling2D(pool_size=(2, 2), input_shape=(64, 64, 3)))
model.add(base_model)
model.add(Flatten())
model.add(Dropout(.25))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(200, activation='softmax'))


model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

history = model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.1)

test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Accuracy (base model on TinyImagenet): {test_accuracy * 100:.2f}%')

Epoch 1/5
1336/1336 [==============================] - 16s 8ms/step - loss: 5.0985 - categorical_accuracy: 0.0221 - val_loss: 4.9346 - val_categorical_accuracy: 0.0344
Epoch 2/5
1336/1336 [==============================] - 9s 7ms/step - loss: 4.9729 - categorical_accuracy: 0.0304 - val_loss: 4.8925 - val_categorical_accuracy: 0.0411
Epoch 3/5
1336/1336 [==============================] - 9s 7ms/step - loss: 4.9369 - categorical_accuracy: 0.0339 - val_loss: 4.8808 - val_categorical_accuracy: 0.0402
Epoch 4/5
1336/1336 [==============================] - 9s 7ms/step - loss: 4.9144 - categorical_accuracy: 0.0352 - val_loss: 4.8699 - val_categorical_accuracy: 0.0398
Epoch 5/5
157/157 [==============================] - 1s 4ms/step - loss: 4.8894 - categorical_accuracy: 0.0404
Test Accuracy (base model on TinyImagenet): 4.04%


In [8]:
model.save('custom_model_tiny_image.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
base_model = tf.keras.models.load_model('/content/model_configuration3.h5')

In [10]:

base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 2, 2, 128)         0

In [11]:
base_model.pop()

In [12]:
# finetunning of the base_model can improve the accuracy.
base_model.trainable=True

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

model = tf.keras.models.Sequential()
model.add(MaxPooling2D(pool_size=(2, 2), input_shape=(64, 64, 3)))
model.add(base_model)
model.add(Flatten())
model.add(Dropout(.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(200, activation='softmax'))

In [14]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="categorical_crossentropy", metrics=["categorical_accuracy"])

history = model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.1)

test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Accuracy (base model on TinyImagenet): {test_accuracy * 100:.2f}%')

Epoch 1/5
1336/1336 [==============================] - 13s 7ms/step - loss: 4.9971 - categorical_accuracy: 0.0271 - val_loss: 4.7298 - val_categorical_accuracy: 0.0502
Epoch 2/5
1336/1336 [==============================] - 9s 7ms/step - loss: 4.6245 - categorical_accuracy: 0.0585 - val_loss: 4.5355 - val_categorical_accuracy: 0.0704
Epoch 3/5
1336/1336 [==============================] - 9s 6ms/step - loss: 4.4438 - categorical_accuracy: 0.0755 - val_loss: 4.3552 - val_categorical_accuracy: 0.0940
Epoch 4/5
1336/1336 [==============================] - 9s 6ms/step - loss: 4.3244 - categorical_accuracy: 0.0918 - val_loss: 4.3223 - val_categorical_accuracy: 0.0965
Epoch 5/5
157/157 [==============================] - 0s 3ms/step - loss: 4.2790 - categorical_accuracy: 0.1068
Test Accuracy (base model on TinyImagenet): 10.68%


In [15]:
model.save('custom_funetunning.h5')